In [404]:
from mc_openapi.doml_mc.intermediate_model.metamodel import parse_metamodel, parse_inverse_associations
import yaml
with open("../assets/doml_meta_v2.0.yaml") as mmf:
    mmdoc = yaml.load(mmf, yaml.Loader)
mm = parse_metamodel(mmdoc)
inv_assoc = parse_inverse_associations(mmdoc)

In [405]:
import prettyprinter as pp
from prettyprinter.prettyprinter import IMPLICIT_MODULES
pp.install_extras(include=['dataclasses'])

In [406]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.metamodel')

### Load the DOML document here:

In [407]:
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_doml_model
from mc_openapi.doml_mc import DOMLVersion
# doml_document_path = "../../tests/doml/nginx-openstack_v2.0.domlx"
doml_document_path = "../../tests/doml/nginx-openstack_v2.0_wrong_vm_iface.domlx"
# doml_document_path = "../../tests/doml/nginx-openstack_v2.0_wrong_iface_uniq.domlx"
with open(doml_document_path, "rb") as xmif:
    doc = xmif.read()

im, _ = parse_doml_model(doc, DOMLVersion.V2_0)
# print(im)

In [408]:
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_xmi_model
doml_xmi = parse_xmi_model(doc, DOMLVersion.V2_0)

In [409]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.doml_element')

In [410]:
from z3 import Solver, DatatypeSortRef, FuncDeclRef, Consts, ForAll, Exists, Implies, And, Or, Not

from mc_openapi.doml_mc.z3encoding.metamodel_encoding import (
    def_association_rel,
    def_attribute_rel,
    mk_association_sort_dict,
    mk_attribute_sort_dict,
    mk_class_sort_dict
)
from mc_openapi.doml_mc.z3encoding.im_encoding import (
    assert_im_associations_q,
    assert_im_attributes,
    def_elem_class_f_and_assert_classes,
    mk_elem_sort_dict,
    mk_stringsym_sort_dict
)
from mc_openapi.doml_mc.z3encoding.types import Refs
from mc_openapi.doml_mc.z3encoding.utils import mk_attr_data_sort



def init(unbound_elems_num: int = 0):

    solver = Solver()

    class_sort, class_ = mk_class_sort_dict(mm, solver.ctx)
    assoc_sort, assoc = mk_association_sort_dict(mm, solver.ctx)
    attr_sort, attr = mk_attribute_sort_dict(mm, solver.ctx)
    ss_sort, ss = mk_stringsym_sort_dict(im, mm, solver.ctx)
    attr_data = mk_attr_data_sort(ss_sort, solver.ctx)

    unbound_elems = [f"unbound{i}" for i in range(unbound_elems_num)]
    elem_sort, elem = mk_elem_sort_dict(im, solver.ctx, unbound_elems)

    elem_class_f = def_elem_class_f_and_assert_classes(
        im,
        solver,
        elem_sort,
        elem,
        class_sort,
        class_
    )
    attr_rel = def_attribute_rel(
        attr_sort,
        elem_sort,
        attr_data
    )
    assert_im_attributes(
        attr_rel,
        solver,
        im,
        mm,
        elem,
        attr_sort,
        attr,
        attr_data,
        ss
    )
    assoc_rel = def_association_rel(
        assoc_sort,
        elem_sort
    )
    assert_im_associations_q(
        assoc_rel,
        solver,
        {k: v for k, v in im.items() if k not in unbound_elems},
        elem,
        assoc_sort,
        assoc,
    )

    # EXAMPLE REQ: All VMs have a network interface.
    vm, iface = Consts(
        "vm iface", elem_sort
    )
    vmIfaceAssertion = ForAll(
        [vm],
        Implies(
            elem_class_f(vm) == class_["infrastructure_VirtualMachine"],
            Exists(
                [iface],
                And(
                    assoc_rel(vm, assoc["infrastructure_ComputingNode::ifaces"], iface)
                )
            )
        )
    )
    solver.assert_and_track(vmIfaceAssertion, "vm_iface")

    # END EXAMPLE REQ

    return (solver,
        unbound_elems, 
        elem_class_f,
        elem_sort,
        elem,
        attr_rel,
        assoc_rel,
        class_sort, class_,
        assoc_sort, assoc,
        attr_sort, attr,
        ss_sort, ss,
        attr_data,
        vmIfaceAssertion
    )

In [411]:
# # nginx has a source code property
# softwareComponent, prop = Consts("softwareComponent prop", elem_sort)
# nginxSourceCode = ForAll(
#     [softwareComponent],
#     Implies(
#         And(
#             elem_class_f(softwareComponent) == class_["application_SoftwareComponent"],
#             attr_rel(softwareComponent, attr["commons_DOMLElement::name"], attr_data.ss(ss["nginx"]))
#         ),
#         Exists(
#             [prop],
#             And(
#                 elem_class_f(prop) == class_["commons_SProperty"],
#                 attr_rel(prop, attr["commons_Property::key"], attr_data.ss(ss["source_code"])),
#                 assoc_rel(softwareComponent, assoc["commons_DOMLElement::annotations"], prop)
#             )
#         )
#     )
# )
# solver.assert_and_track(nginxSourceCode, "nginx_source_code")

In [412]:
def im_print(a):
    
    print(ret_str)

In [413]:
(   solver,
    unbound_elems, 
    elem_class_f, 
    elem_sort, 
    elem, 
    attr_rel, 
    assoc_rel, 
    class_sort, 
    class_,
    assoc_sort, assoc,
    attr_sort, attr,
    ss_sort, ss,
    attr_data,
    vmIfaceAssertion
) = init(unbound_elems_num=0)

In [414]:
solver.check()

unsat

In [415]:
solver.unsat_core()

[associations elem_140550376150000 elem_140550365458448,
 vm_iface,
 associations elem_140550376150000 elem_140550365461040,
 associations elem_140550376150000 elem_140550387749920,
 associations elem_140550376150000 elem_140550552661552,
 associations elem_140550376150000 elem_140550552660688,
 associations elem_140550376150000 elem_140550552661264,
 associations elem_140550376150000 elem_140550387760480,
 associations elem_140550376150000 elem_140550376146016,
 associations elem_140550376150000 elem_140550387986160,
 associations elem_140550376150000 elem_140550376150000,
 associations elem_140550376150000 elem_140550376151344,
 associations elem_140550376150000 elem_140550555422576,
 associations elem_140550376150000 elem_140550365462192,
 associations elem_140550376150000 elem_140550365460464,
 associations elem_140550376150000 elem_140550555423440,
 associations elem_140550376150000 elem_140550365458496,
 associations elem_140550376150000 elem_140550387752608,
 associations elem_140550376150000 elem_140550365459264,
 associations elem_140550376150000 elem_140550365457104,
 associations elem_140550376150000 elem_140550552660592,
 associations elem_140550376150000 elem_140550365461520,
 associations elem_140550376150000 elem_140550537624352,
 associations elem_140550376150000 elem_140550556240720,
 associations elem_140550376150000 elem_140550552660496,
 associations elem_140550376150000 elem_140550376151632,
 elem_class elem_140550376150000 infrastructure_VirtualMachine,
 associations elem_140550376150000 elem_140550365461280]

In [416]:
try:
    model = solver.model()
except:
    print("No model available because it was unsat, trying again with one more unbound variable")
# print(model)
# print(model[vm])
# print(model[iface])
# pprint([im_print(x) for x in model])

No model available because it was unsat, trying again with one more unbound variable


In [417]:
(   solver,
    unbound_elems, 
    elem_class_f, 
    elem_sort, 
    elem, 
    attr_rel, 
    assoc_rel, 
    class_sort, 
    class_,
    assoc_sort, assoc,
    attr_sort, attr,
    ss_sort, ss,
    attr_data,
    vmIfaceAssertion
) = init(unbound_elems_num=1)

In [418]:
solver.check()

sat

In [419]:
solver.unsat_core()

[]

In [420]:
model = solver.model()
# print(model)
# print(model[vm])
# print(model[iface])
# pprint([im_print(x) for x in model])

In [421]:
from itertools import product

def get_unbound_vars_results(model):
    return [ ((e1n, e1), a, (e2n, e2)) 
        for (e1n, e1), a, (e2n, e2) in product(elem.items(), assoc.values(), elem.items()) 
        if (e1n in unbound_elems or e2n in unbound_elems) and model.eval(assoc_rel(e1, a, e2))
    ]

def get_uvars_as_text(res):
    ret = []
    for (e1n, e1), a, (e2n, e2) in res:
        tokens = str(str(e1) + " " + str(a) + " " + str(e2)).split()
        ret_str = ""
        for token in tokens:
            if token[0:4] == "elem":
                value = im.get(token)
                ret_str += f"{value.class_} ({value.user_friendly_name})" if value else f"<'{token}' not found>"
            else:
                ret_str += token
            ret_str += " "
        ret.append(ret_str.strip())
    return ret

In [422]:
res1 = get_unbound_vars_results(model)

len(res1)

34

In [423]:
res1_text = "\n".join(get_uvars_as_text(res1))
print(res1_text)
print(len(res1))

infrastructure_VirtualMachine (vm1) commons_Property::reference unbound0
infrastructure_VirtualMachine (vm1) application_SoftwareComponent::exposedInterfaces unbound0
infrastructure_VirtualMachine (vm1) application_SoftwareComponent::consumedInterfaces unbound0
infrastructure_VirtualMachine (vm1) application_SaaS::exposedInterfaces unbound0
infrastructure_VirtualMachine (vm1) infrastructure_InfrastructureLayer::nodes unbound0
infrastructure_VirtualMachine (vm1) infrastructure_InfrastructureLayer::storages unbound0
infrastructure_VirtualMachine (vm1) infrastructure_InfrastructureLayer::faas unbound0
infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::ifaces unbound0
infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::location unbound0
infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::group unbound0
infrastructure_VirtualMachine (vm1) infrastructure_Container::generatedFrom unbound0
infrastructure_VirtualMachine (vm1) infrastructure_Container::

In [424]:
unbound_elems_ref = Consts(" ".join(unbound_elems), elem_sort)
print(unbound_elems)
for ue in unbound_elems_ref:
    print(model[ue])

['unbound0']
None


In [425]:
p1 = res1[0]
(e1n, e1), a, (e2n, e2) = p1

solver.assert_and_track(Not(assoc_rel(e1, a, e2)), "ub_1")

In [426]:
solver.check()

sat

In [427]:
model = solver.model()
res2 = get_unbound_vars_results(model)
res2_text = print("\n".join(get_uvars_as_text(res2)))

print(res2_text)
print(len(res2))

infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::ifaces unbound0
None
1


In [428]:
from difflib import HtmlDiff
from IPython.display import display, HTML

diff = HtmlDiff().make_table(get_uvars_as_text(res1), get_uvars_as_text(res2))
display(HTML(diff))

n,1,infrastructure_VirtualMachine (vm1) commons_Property::reference unbound0,n,,
,2,infrastructure_VirtualMachine (vm1) application_SoftwareComponent::exposedInterfaces unbound0,,,
,3,infrastructure_VirtualMachine (vm1) application_SoftwareComponent::consumedInterfaces unbound0,,,
,4,infrastructure_VirtualMachine (vm1) application_SaaS::exposedInterfaces unbound0,,,
,5,infrastructure_VirtualMachine (vm1) infrastructure_InfrastructureLayer::nodes unbound0,,,
,6,infrastructure_VirtualMachine (vm1) infrastructure_InfrastructureLayer::storages unbound0,,,
,7,infrastructure_VirtualMachine (vm1) infrastructure_InfrastructureLayer::faas unbound0,,,
,8,infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::ifaces unbound0,,1,infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::ifaces unbound0
t,9,infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::location unbound0,t,,
,10,infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::group unbound0,,,
,11,infrastructure_VirtualMachine (vm1) infrastructure_Container::generatedFrom unbound0,,,
